In [10]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import requests

In [9]:
! mkdir data

In [2]:
df = pd.read_csv('../../util/airport_w_station.csv')

In [3]:
df["done"]=np.zeros(len(df))

In [6]:
df.columns

Index(['IATA', 'STATION', 'done'], dtype='object')

In [11]:
Token = 'YHXsBwZbcfQIEKbFRRGVqMQSnPTAWFgt'

In [12]:
i = 0
for row in df.values:
    iata = row[0]
    station_code = row[1]
    r = requests.get('https://www.ncei.noaa.gov/access/services/data/v1?dataset=daily-summaries&dataTypes=TMAX,TMIN,PRCP&stations='+station_code+'&startDate=2013-01-01&endDate=2013-12-31&units=metric', headers={'token':Token})
    r_str = r.content.decode("utf-8")
    # write to file named station_code.csv
    with open('data/'+station_code+'.csv', 'w') as f:
        f.write(r_str)
        
    df["done"][i]=1
    i+=1

/var/folders/tx/5_6_zg1d0fd4zjwzxg1ztrth0000gn/T/ipykernel_24828/2817875451.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["done"][i]=1


In [14]:
# read all csv files in data folder and create a single dataframe
df_all = pd.DataFrame()
for station_code in df["STATION"].values:
    df_station = pd.read_csv('data/'+station_code+'.csv')
    df_all = pd.concat([df_all, df_station], axis=0)


In [21]:
df_tmp = df_all.groupby('STATION').count()
# order by number of days
df_tmp = df_tmp.sort_values(by='DATE', ascending=True)
df_tmp.head()

,DATE,PRCP,TMAX,TMIN
STATION,,,,
USC00509919,1,1,1,1
USW00024166,9,9,9,9
US1PACN0017,23,23,0,0
USC00103940,107,97,106,106
USW00024012,164,162,163,163


In [25]:
df_tmp['coll']=df_tmp['DATE']+df_tmp['TMAX']+df_tmp['TMIN']+df_tmp['PRCP']

In [33]:
# select the rows with coll != 365*3
df_redo = df_tmp[df_tmp['coll']!=1460]

In [35]:
# get the row in df there STATION is in df_redo
df_redo = df[df['STATION'].isin(df_redo.index)]

In [36]:
df_redo

,IATA,STATION,done
1,PSE,RQC00667292,1.0
3,DLG,USR0000AMOT,1.0
8,SNA,USW00093184,1.0
12,IDA,USW00024145,1.0
14,JLN,USW00013987,1.0
15,PSG,USW00025329,1.0
19,MYR,NOE00133446,1.0
20,OAK,USW00023230,1.0
25,COD,USC00481840,1.0
26,BTM,USW00024135,1.0


In [37]:
# reset index
df_redo = df_redo.reset_index(drop=True)

In [38]:
df_redo

,IATA,STATION,done
0,PSE,RQC00667292,1.0
1,DLG,USR0000AMOT,1.0
2,SNA,USW00093184,1.0
3,IDA,USW00024145,1.0
4,JLN,USW00013987,1.0
5,PSG,USW00025329,1.0
6,MYR,NOE00133446,1.0
7,OAK,USW00023230,1.0
8,COD,USC00481840,1.0
9,BTM,USW00024135,1.0
